# Lucas Tree

## Question 1

In [ ]:
import pickle
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.stats as sts

import pandas_datareader.data as web

In [ ]:
start = datetime.datetime(1972, 1, 1)
end = datetime.datetime(2010, 12, 31)

consumption = web.DataReader('PCECC96', 'fred', start, end) 
market = web.DataReader('WILL5000INDFC', 'fred', start, end) 
t_bill = web.DataReader('TB3MS', 'fred', start, end)         
deflator = web.DataReader('GDPDEF', 'fred', start, end)      


df = pd.concat([consumption, market, deflator, t_bill], sort=True)
df = df.groupby(pd.PeriodIndex(df.index, freq='Q'), axis=0).mean() # convert to quarterly

df.columns = ['consumption', 'market', 'deflator', 't_bill']
df['rMarket'] = df.market / (df.deflator*100)
df['rTbill'] = df.t_bill / (df.deflator*100)


### not sure how to deflate tbill
df.head()

Assume that 𝛽 = 0.99 (since this is quarterly data, this implies an annual risk- free rate of about 4%). Calculate empirical analogs of the expected values and covariance in (5). Using scipy.optimize.broyden1, solve for the value of 𝛾 that leads to equation (5) holding.


In [ ]:
class GMM_Lucas( object ):   
    def __init__(self, data=df, options='q3'):
        
        self.data = data
        self.options = options
                
        R = self.data.rMarket / self.data.rMarket.shift(1)
        self.R = R.dropna()
        
    def estimate_m(self, gamma, beta):
        ratio = self.data.consumption / self.data.consumption.shift(1)
        
        MUC = ratio ** -gamma     
        m = beta * MUC
        return m[1:]
 
    def objective(self, params):
        if params.size == 1:
            gamma = params
            beta = 0.99
            
        if self.options == 'q1':
            m = self.estimate_m(gamma, beta)
            cov = np.cov(m, self.R)[0,1]              # var/cov matrix, take COV
            error = cov + m.mean() * self.R.mean() - 1
            return error
        
        elif self.options == 'q3':        
            m = self.estimate_m(gamma, beta)
            exp = self.R * m
            error = exp.mean() - 1
            return error
        
        elif self.options == 'q4':
            beta, gamma = params
            m = self.estimate_m(gamma, beta)
            cov = np.cov(m, self.R)[0,1]
            err1 = cov + m.mean() * self.R.mean() - 1

            exp = self.R * m
            err2 = exp.mean() - 1
            
            e = np.array([err1, err2])
            return e.T @ np.eye(2) @ e

                   
    def estimate_gamma(self):
        if self.options == 'q4':
            xguess = np.ones(2)
            result = opt.minimize(self.objective, xguess)
        else:
            xguess = np.ones(1)
            ### Non=linear solver gets very different answer to Natasha, come back
            result = opt.broyden1(self.objective, xguess)
        return result
    
    def plot(self):
        n = 60
        N = 5
        γ_grid = np.linspace(0, 5, n)
        β_grid = np.linspace(0.95, 0.99, N)

        cons = self.data.rMarket
        tbill = self.data.rTbill

        gmm_values = np.empty((n, N))
        for i in range(n):
            for j in range(N):
                param = np.array( [ γ_grid[i], β_grid[j] ])
                gmm_values[i, j] = self.objective(param)
                
        plt.figure(figsize=(10, 8))
        for j in range(N):
            plt.plot(γ_grid, gmm_values[:, j], label=f'{β_grid[j]:.2f}')
        plt.title('Value of GMM objective function')
        plt.xlabel('$\gamma$')
        plt.legend()
        plt.show()

In [ ]:
Question1 = GMM_Lucas(data=df, options='q1')
print("Gamma = ",Question1.estimate_gamma())

## Question 2

I would want to try this on a different data set or a different subset of our data to see if gamma is robust to different specifications. I would also check to see whether or not using a different equation to estimate yields the same results. 

## Question 3

In [ ]:
Question3 = GMM_Lucas(data=df, options='q3')
print("Gamma = ",Question3.estimate_gamma())

## Question 4

In [ ]:
Question4 = GMM_Lucas(data=df, options='q4')
ans = Question4.estimate_gamma().x
print("Beta  = ", ans[0], "\nGamma = ", ans[1])

print("\nThis is super interesting. When I allow beta to fluctuate, this provides a much lower value gamma because"
     + " beta can explain some of the savings. ")

Question4.plot()

# Kyle Model

## Question 1

In [ ]:
spy = pickle.load(open('data/SPY', 'rb'))
tho = pickle.load(open('data/THO', 'rb'))
gbx = pickle.load(open('data/GBX', 'rb'))
aapl = pickle.load(open('data/AAPL', 'rb'))
C = pickle.load(open('data/C', 'rb'))

In [ ]:
class OrderBook(object):
    def __init__(self, data, name=None):
        self.name = name
        self.time = data[0]
        self.order = self.clean_orderbook(data[1])
        
        self.prices = np.array(list(self.order.keys()))
        self.quant = np.array(list(self.order.values()))
    
    def clean_orderbook(self,data):
        clean = {ii: jj[0] for ii,jj in data.items()}
        return clean
        
    def regress(self):
        
        beta_hat, gamma_hat, r, p, stderr = sts.linregress(self.prices, self.quant)
        self.beta = beta_hat
        self.gamma = gamma_hat
        return beta_hat, gamma_hat
    
    def plot(self):
        self.regress()
        domain = np.linspace(-100,2500,1000)
        
        fig, ax = plt.subplots(1,1, figsize=(8,6))
        plt.scatter(self.prices, self.quant, alpha=0.6, label='Data Points')
        plt.plot(domain, self.gamma + self.beta*domain, 'r--', label="Estimate")
        plt.title(self.name)
        plt.xlabel("Price")
        plt.ylabel("Quantity")
        plt.xlim(xmin=-100, xmax=2500)
        plt.legend()
        plt.show()

In [ ]:
stocks = [spy, tho, gbx, aapl, C]
timeOday = [0, -1, 150]

books = []

for stock in stocks:
    for time in timeOday:
        books.append(OrderBook(stock[time], '{}-{}'.format(stock,time)))

In [ ]:
books[-1].plot()